**Configuração e Autenticação**

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery

import pandas as pd

**Conexão Big Query**

In [ ]:
project_id = 'case-gb-elcio'
dataset_id = 'vendas'

In [ ]:
client = bigquery.Client(project=project_id)

In [ ]:
def upload_to_bq(df, table_name):
    job_config = bigquery.LoadJobConfig(
        autodetect=True
    )

    job = client.load_table_from_dataframe(
        df, f'{project_id}.{dataset_id}.{table_name}', job_config=job_config
    )
    job.result()

    print(f"Tabela {table_name} carregada com sucesso!")

**Leitura e tratamento das tabelas**

In [ ]:
df_pd_tb_vendas_2017 = client.query(f"SELECT * FROM `case-gb-elcio.vendas.tb_vendas_2017`").to_dataframe()
df_pd_tb_vendas_2018 = client.query(f"SELECT * FROM `case-gb-elcio.vendas.tb_vendas_2018`").to_dataframe()
df_pd_tb_vendas_2019 = client.query(f"SELECT * FROM `case-gb-elcio.vendas.tb_vendas_2019`").to_dataframe()

In [ ]:
df_pd_unificado = pd.concat([df_pd_tb_vendas_2017, df_pd_tb_vendas_2018, df_pd_tb_vendas_2019], ignore_index=True)

print("Tamanho do DataFrame unificado:", df_pd_unificado.shape)

print(df_pd_unificado.head())

Tamanho do DataFrame unificado: (3000, 6)
   ID_MARCA MARCA  ID_LINHA  LINHA DATA_VENDA  QTD_VENDA
0         4  VULT         4  SOLAR 2019-03-06         15
1         4  VULT         4  SOLAR 2019-03-08         15
2         4  VULT         4  SOLAR 2019-07-23          5
3         4  VULT         4  SOLAR 2019-06-22         12
4         4  VULT         4  SOLAR 2019-11-11          5


In [ ]:
df_pd_unificado['ANO'] = df_pd_unificado['DATA_VENDA'].dt.year
df_pd_unificado['MES'] = df_pd_unificado['DATA_VENDA'].dt.month

In [ ]:
df_consolidado_vendas_ano_mes = df_pd_unificado.groupby(['ANO', 'MES'])['QTD_VENDA'].sum().reset_index(name='TOTAL_VENDAS')

df_consolidado_vendas_ano_mes = df_consolidado_vendas_ano_mes.sort_values(by=['ANO', 'MES'])

print(df_consolidado_vendas_ano_mes)

     ANO  MES  TOTAL_VENDAS
0   2017    1           832
1   2017    2           739
2   2017    3          1015
3   2017    4           954
4   2017    5           679
5   2017    6           785
6   2017    7           670
7   2017    8           810
8   2017    9           908
9   2017   10          1137
10  2017   11           919
11  2017   12           887
12  2019    1          1630
13  2019    2          1566
14  2019    3          1416
15  2019    4          1988
16  2019    5          1896
17  2019    6          1734
18  2019    7          1546
19  2019    8          2056
20  2019    9          2030
21  2019   10          1752
22  2019   11          1624
23  2019   12          1998


In [ ]:
df_consolidado_vendas_marca_linha = df_pd_unificado.groupby(['MARCA', 'LINHA'])['QTD_VENDA'].sum().reset_index(name='TOTAL_VENDAS')

df_consolidado_vendas_marca_linha = df_consolidado_vendas_marca_linha.sort_values(by=['MARCA', 'LINHA'])

print(df_consolidado_vendas_marca_linha)

                  MARCA        LINHA  TOTAL_VENDAS
0         BELEZA NA WEB      CABELOS          1039
1         BELEZA NA WEB  HIDRATANTES          1073
2         BELEZA NA WEB    MAQUIAGEM          1310
3         BELEZA NA WEB   PERFUMARIA          1043
4         BELEZA NA WEB        SOLAR          1127
5             BOTICÁRIO      CABELOS          1534
6             BOTICÁRIO  HIDRATANTES          1233
7             BOTICÁRIO    MAQUIAGEM          1058
8             BOTICÁRIO   PERFUMARIA          1112
9             BOTICÁRIO        SOLAR          1271
10               EUDORA      CABELOS          1259
11               EUDORA  HIDRATANTES          1101
12               EUDORA    MAQUIAGEM          1247
13               EUDORA   PERFUMARIA          1373
14               EUDORA        SOLAR          1348
15  QUEM DISSE BERENICE      CABELOS           960
16  QUEM DISSE BERENICE  HIDRATANTES          1313
17  QUEM DISSE BERENICE    MAQUIAGEM          1367
18  QUEM DISSE BERENICE   PERFU

In [ ]:
df_consolidado_vendas_marca_ano_mes = df_pd_unificado.groupby(['MARCA', 'ANO', 'MES'])['QTD_VENDA'].sum().reset_index(name='TOTAL_VENDAS')

df_consolidado_vendas_marca_ano_mes = df_consolidado_vendas_marca_ano_mes.sort_values(by=['MARCA', 'ANO', 'MES'])

print(df_consolidado_vendas_marca_ano_mes)

             MARCA   ANO  MES  TOTAL_VENDAS
0    BELEZA NA WEB  2017    1           207
1    BELEZA NA WEB  2017    2           175
2    BELEZA NA WEB  2017    3           219
3    BELEZA NA WEB  2017    4           108
4    BELEZA NA WEB  2017    5           133
..             ...   ...  ...           ...
115           VULT  2019    8           420
116           VULT  2019    9           416
117           VULT  2019   10           322
118           VULT  2019   11           422
119           VULT  2019   12           482

[120 rows x 4 columns]


In [ ]:
df_consolidado_vendas_linha_ano_mes = df_pd_unificado.groupby(['LINHA', 'ANO', 'MES'])['QTD_VENDA'].sum().reset_index(name='TOTAL_VENDAS')

df_consolidado_vendas_linha_ano_mes = df_consolidado_vendas_linha_ano_mes.sort_values(by=['LINHA', 'ANO', 'MES'])

print(df_consolidado_vendas_linha_ano_mes)

       LINHA   ANO  MES  TOTAL_VENDAS
0    CABELOS  2017    1           120
1    CABELOS  2017    2           189
2    CABELOS  2017    3           203
3    CABELOS  2017    4           181
4    CABELOS  2017    5           108
..       ...   ...  ...           ...
115    SOLAR  2019    8           392
116    SOLAR  2019    9           440
117    SOLAR  2019   10           402
118    SOLAR  2019   11           410
119    SOLAR  2019   12           494

[120 rows x 4 columns]


**Salvando tabelas no BigQuery**

In [ ]:
upload_to_bq(df_consolidado_vendas_ano_mes, 'tb_consolidado_vendas_ano_mes')
upload_to_bq(df_consolidado_vendas_marca_linha, 'tb_consolidado_vendas_marca_linha')
upload_to_bq(df_consolidado_vendas_marca_ano_mes, 'tb_consolidado_vendas_marca_ano_mes')
upload_to_bq(df_consolidado_vendas_linha_ano_mes, 'tb_consolidado_vendas_linha_ano_mes')

Tabela tb_consolidado_vendas_ano_mes carregada com sucesso!
Tabela tb_consolidado_vendas_marca_linha carregada com sucesso!
Tabela tb_consolidado_vendas_marca_ano_mes carregada com sucesso!
Tabela tb_consolidado_vendas_linha_ano_mes carregada com sucesso!
